In [2]:
# 📊 Optimized MACD Trading Strategy vs. Buy & Hold

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

# Simulated S&P 500-futures data
np.random.seed(42)
dates = pd.date_range(start="2022-01-01", periods=500, freq="D")
price = np.cumsum(np.random.randn(500) * 10) + 4000  # Simulated price movement

df = pd.DataFrame(index=dates)
df["Close"] = price

df["High"] = df["Close"] + np.random.rand(len(df)) * 10  # Simulated High

df["Low"] = df["Close"] - np.random.rand(len(df)) * 10  # Simulated Low

# MACD Calculation with Optimized Parameters
df["12_EMA"] = df["Close"].ewm(span=8, adjust=False).mean()
df["26_EMA"] = df["Close"].ewm(span=21, adjust=False).mean()
df["MACD"] = df["12_EMA"] - df["26_EMA"]
df["Signal_Line"] = df["MACD"].ewm(span=6, adjust=False).mean()

# Trading Signals
df["Position"] = 0
df.loc[df["MACD"] > df["Signal_Line"], "Position"] = 1  # Buy Signal
df.loc[df["MACD"] < df["Signal_Line"], "Position"] = -1  # Sell Signal
df["Trade_Signal"] = df["Position"].diff()

# Stop-Loss & Take-Profit Implementation
stop_loss_pct = 0.05  # 5% stop-loss
take_profit_pct = 0.10  # 10% take-profit

# Volatility Filter (ATR - Average True Range)
df["ATR"] = (df["High"] - df["Low"]).rolling(window=14).mean()

def is_trade_allowed(index):
    return df["ATR"].iloc[index] > df["ATR"].rolling(30).mean().iloc[index]

# Backtesting Strategy
initial_cash = 10000  # Start capital
cash = initial_cash
shares = 0
portfolio_value = []
entry_price = 0

for i in range(len(df)):
    price = df["Close"].iloc[i]
    
    if df["Trade_Signal"].iloc[i] == 1 and is_trade_allowed(i):  # Buy Signal with ATR filter
        shares = cash // price
        cash -= shares * price
        entry_price = price  # Store entry price

    elif df["Trade_Signal"].iloc[i] == -1 and shares > 0:  # Sell Signal
        cash += shares * price
        shares = 0
        entry_price = 0
    
    # Stop-Loss & Take-Profit Logic
    if shares > 0:
        if price < entry_price * (1 - stop_loss_pct):  # Stop-Loss Trigger
            cash += shares * price
            shares = 0
            entry_price = 0
        elif price > entry_price * (1 + take_profit_pct):  # Take-Profit Trigger
            cash += shares * price
            shares = 0
            entry_price = 0
    
    portfolio_value.append(cash + shares * price)

df["MACD_Strategy_Value"] = portfolio_value

# Buy & Hold Benchmark
df["Buy_Hold_Value"] = initial_cash * (df["Close"] / df["Close"].iloc[0])

# 📊 Comparison of MACD Trading vs. Buy & Hold
final_macd_value = df["MACD_Strategy_Value"].iloc[-1]
final_buy_hold_value = df["Buy_Hold_Value"].iloc[-1]

macd_profit = final_macd_value - initial_cash
buy_hold_profit = final_buy_hold_value - initial_cash

macd_return = (macd_profit / initial_cash) * 100
buy_hold_return = (buy_hold_profit / initial_cash) * 100

# Generate Comparison Report
comparison_report = f"""
{"="*50}
📈 **Comparison of MACD Trading vs. Buy & Hold**
{"="*50}

🔹 **MACD Trading Strategy:**
   - **Final Value:** ${final_macd_value:,.2f}
   - **Total Profit:** ${macd_profit:,.2f}
   - **Return:** {macd_return:.2f}%

🔹 **Buy & Hold (S&P 500 Benchmark):**
   - **Final Value:** ${final_buy_hold_value:,.2f}
   - **Total Profit:** ${buy_hold_profit:,.2f}
   - **Return:** {buy_hold_return:.2f}%

📊 **Conclusion:**
{"✅ MACD Trading outperformed the market!" if macd_return > buy_hold_return else "❌ MACD Trading underperformed the market!"}
{"="*50}
"""

# Print the updated comparison report
print(comparison_report)



📈 **Comparison of MACD Trading vs. Buy & Hold**

🔹 **MACD Trading Strategy:**
   - **Final Value:** $10,000.00
   - **Total Profit:** $0.00
   - **Return:** 0.00%

🔹 **Buy & Hold (S&P 500 Benchmark):**
   - **Final Value:** $10,072.97
   - **Total Profit:** $72.97
   - **Return:** 0.73%

📊 **Conclusion:**
❌ MACD Trading underperformed the market!

